In [32]:
%pip install scikit-learn


DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import re
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from gensim.parsing.preprocessing import STOPWORDS, strip_tags, strip_numeric, strip_punctuation, strip_multiple_whitespaces, remove_stopwords, strip_short, stem_text
import numpy as np
import sys
sys.path.append('../')
from utils import remove_similar_rows,remove_similar_rows_per_player, find_lines_with_player


In [2]:
url = 'https://raw.githubusercontent.com/svisel22/SS23-BIPM-Analytics-Lab---Group-4-repository/main/data_files/all_data_v3.csv'
df = pd.read_csv(url)


In [3]:
# Filter out the German data and reindex
df_ger = df[df["language"] == "de"]
df_ger = df_ger.reset_index(drop=True)

In [4]:
df = remove_similar_rows_per_player(df_ger, df_ger['player'].unique())


In [6]:
df_ger

,data,player,language,publishedAt
0,{'content': 'Trainer Xabi Alonso vor den mitge...,Exequiel Palacios,de,2023-02-24T09:33:31Z
1,{'content': 'Nutze kicker auf seinen digitalen...,Exequiel Palacios,de,2023-03-03T21:35:13Z
2,{'content': 'Nutze kicker auf seinen digitalen...,Exequiel Palacios,de,2023-03-07T11:34:39Z
3,{'content': 'Mitchel Bakker am Mittwoch im Abs...,Exequiel Palacios,de,2023-03-08T14:25:18Z
4,{'content': '© G+J Medien GmbH\nZugang zu alle...,Exequiel Palacios,de,2023-03-09T19:36:53Z
...,...,...,...,...
566,{'content': 'Warum sehe ich FAZ.NET nicht?\nSe...,Piero Hincapié,de,2023-02-17T07:51:22Z
567,{'content': 'Warum sehe ich FAZ.NET nicht?\nSe...,Piero Hincapié,de,2023-02-17T07:51:22Z
568,{'content': '© G+J Medien GmbH\nZugang zu alle...,Piero Hincapié,de,2023-02-17T06:22:51Z
569,{'content': '© G+J Medien GmbH\nZugang zu alle...,Piero Hincapié,de,2023-02-17T05:22:48Z


In [5]:
df

,data,player,language,publishedAt
0,{'content': 'Trainer Xabi Alonso vor den mitge...,Exequiel Palacios,de,2023-02-24T09:33:31Z
1,{'content': 'Nutze kicker auf seinen digitalen...,Exequiel Palacios,de,2023-03-03T21:35:13Z
2,{'content': 'Nutze kicker auf seinen digitalen...,Exequiel Palacios,de,2023-03-07T11:34:39Z
3,{'content': 'Mitchel Bakker am Mittwoch im Abs...,Exequiel Palacios,de,2023-03-08T14:25:18Z
4,{'content': '© G+J Medien GmbH\nZugang zu alle...,Exequiel Palacios,de,2023-03-09T19:36:53Z
...,...,...,...,...
31,"{'content': ""Der SC Freiburg und Bayer Leverku...",Piero Hincapié,de,2023-02-26T16:28:57Z
33,{'content': 'Ein Monaco-Betreuer packt Sardar ...,Piero Hincapié,de,2023-02-22T16:59:33Z
34,{'content': 'Leverkusens Torwart Lukas Hradeck...,Piero Hincapié,de,2023-02-17T17:14:46Z
35,{'content': 'Das Wichtigste\n\n\nFast alle Spi...,Piero Hincapié,de,2023-02-17T08:56:42Z


In [37]:
# Remove the similiar rows (The Function is imported from utils on top)

data_lower = df.copy()

data_lower['data'] = data_lower['data'].str.lower()
data_lower['player'] = data_lower['player'].str.lower()
data_lower.head()

data_wo_pattern = data_lower.copy()

data_wo_pattern['data'] = data_wo_pattern['data'].apply(lambda x: re.sub(r"^{\'content\': \'", "", str(x)))
data_wo_pattern['data'] = data_wo_pattern['data'].apply(lambda x: re.sub(r"\\nAdvertisement\\nAdvertisement\\nAdvertisement|nutze kicker'\}$", "", str(x)))


In [38]:
#data_wo_pattern['data'] = data_wo_pattern['data'].apply(lambda x: re.sub(r'^\s*nutze kicker.*?\.', "", str(x), flags=re.M))


In [39]:
data_wo_pattern['data'].iloc[1]

'nutze kicker auf seinen digitalen plattformen wie gewohnt mit werbung und tracking. die zustimmung kann jederzeit für die zukunft widerrufen werden. details zu werbe- und analyse-trackern findest du in unserer datenschutzerklärung oder bei cookies & tracking am ende jeder seite.\\nmit dem pur-abo nutzt du kicker auf seinen digitalen plattformen ohne werbetracking und praktisch werbefrei.\\nnur €2,49 im monat\\nbereits pur-abonnent? hier anmelden.\\nalle antworten zum pur-abo findest du hier.\\nwerbung und tracking: um dir relevante inhalte und personalisierte werbung anzubieten, setzen wir cookies und andere technologien ein. damit messen wir, wie und womit du unsere angebote nutzt. diese daten geben wir auch an dritte weiter. wir greifen dabei auf dein endgerät zu, speichern cookies oder sonstige informationen und wir oder dritte können mit diesen sowie mit persönlichen identifikatoren (z.b. geräte-kennungen oder ip-adressen) und basierend auf deinem individuellen nutzungsverhalten .

## Find sentences with Playername

In [40]:
data_wo_pattern

,data,player,language,publishedAt
0,trainer xabi alonso vor den mitgereisten fans ...,exequiel palacios,de,2023-02-24T09:33:31Z
1,nutze kicker auf seinen digitalen plattformen ...,exequiel palacios,de,2023-03-03T21:35:13Z
2,nutze kicker auf seinen digitalen plattformen ...,exequiel palacios,de,2023-03-07T11:34:39Z
3,mitchel bakker am mittwoch im abschluss-traini...,exequiel palacios,de,2023-03-08T14:25:18Z
4,© g+j medien gmbh\nzugang zu allen stern plus-...,exequiel palacios,de,2023-03-09T19:36:53Z
...,...,...,...,...
31,"{'content': ""der sc freiburg und bayer leverku...",piero hincapié,de,2023-02-26T16:28:57Z
33,ein monaco-betreuer packt sardar azmoun (l.) a...,piero hincapié,de,2023-02-22T16:59:33Z
34,leverkusens torwart lukas hradecky sitzt fluch...,piero hincapié,de,2023-02-17T17:14:46Z
35,das wichtigste\n\n\nfast alle spieler waren be...,piero hincapié,de,2023-02-17T08:56:42Z


In [41]:
data_with_playernames = find_lines_with_player(data_wo_pattern, data_wo_pattern['player'].unique())

In [42]:
data_with_playernames

,data,player,language,publishedAt
0,glücksmoment! exequiel palacios hebt nach sein...,exequiel palacios,de,2023-02-24T09:33:31Z
1,dafür erscheint seine verletzungspause schon z...,exequiel palacios,de,2023-03-03T21:35:13Z
2,wenn bayer 04 am donnerstag im achtelfinal-hin...,exequiel palacios,de,2023-03-07T11:34:39Z
3,"nicht ausgeschlossen ist jedoch, dass bakker g...",exequiel palacios,de,2023-03-08T14:25:18Z
4,,exequiel palacios,de,2023-03-09T19:36:53Z
...,...,...,...,...
21,"bayer, das ohne die gesperrten jonathan tah, p...",piero hincapié,de,2023-02-26T16:28:57Z
22,vor allem die provokationen von monacos krépin...,piero hincapié,de,2023-02-22T16:59:33Z
23,grund war offenbar monacos 2:2-torschütze diat...,piero hincapié,de,2023-02-17T17:14:46Z
24,tut es in der kollektivität aber dann doch sel...,piero hincapié,de,2023-02-17T08:56:42Z


In [43]:
data_with_playernames['data'].iloc[5]

'allerdings waren in robert andrich und dem argentinischen weltmeister exequiel palacios beide eigentlich gesetzten sechser gesperrt, charles aranguiz ist zudem verletzt. am donnerstag machten sie ihre sache zumindest ordentlich.'

## Pattern Regex


In [44]:
patternlist = [
    r'^\s*nutze kicker.*?\.',
    r'^\s*die zustimmung.*?\.',
    r'^\s*details zu werbe-.*?\.',
    r'^\s*\\nmit dem.*?\.',
    r'^\s*\\nnur €2,49.*?\.',
    r'^\s*\\nbereits pur-.*?\.',
    r'^\s*\\nalle antworten.*?\.',
    r'^\s*\\nwerbung.*?\.',
    r'^\s*damit messen.*?\.',
    r'^\s*diese daten .*?\.', 
    r'^\s*wir greifen .*?\.', 
    r'^\s*b..*?\.', 
    r'^\s*\\nhinweis.*?\.',
    r'^\s*..\\nhinweis.*?\.',
    r'^\s*die usa.*?\.',
    r'^\s*es besteht.*?\.',    
    # add more patterns here
]

In [45]:
for pattern in patternlist:

    data_wo_pattern['data'] = data_wo_pattern['data'].apply(lambda x: re.sub(pattern, "", str(x), flags=re.M))


## Stopwords

In [46]:
german_stop_words = stopwords.words('german')

# Create a list of stop words to remove 
stop_words_to_remove = ['nicht', 'nichts']

# Remove the stop words to remove from the stop words list
for word in stop_words_to_remove:
  german_stop_words.remove(word)

data_rm = df_ger.copy()

# Define a function to apply remove_stopwords on a column
def remove_stopwords_from_text(text):
    return remove_stopwords(text, stopwords=german_stop_words)

# Apply the remove_stopwords function to the 'text' column using the apply method
data_rm['data'] = data_rm['data'].apply(remove_stopwords_from_text)


In [47]:
data_rm.head()

,data,player,language,publishedAt
0,{'content': 'Trainer Xabi Alonso mitgereisten ...,Exequiel Palacios,de,2023-02-24T09:33:31Z
1,{'content': 'Nutze kicker digitalen Plattforme...,Exequiel Palacios,de,2023-03-03T21:35:13Z
2,{'content': 'Nutze kicker digitalen Plattforme...,Exequiel Palacios,de,2023-03-07T11:34:39Z
3,{'content': 'Mitchel Bakker Mittwoch Abschluss...,Exequiel Palacios,de,2023-03-08T14:25:18Z
4,{'content': '© G+J Medien GmbH\nZugang STERN P...,Exequiel Palacios,de,2023-03-09T19:36:53Z
